In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from pathlib import Path
import keras
import tensorflow as tf 

In [7]:
train_dir = Path('brain tumor/Training/')
test_dir = Path('brain tumor/Testing/')

from keras.preprocessing.image import ImageDataGenerator

height=224 # images ont des tailles différentes donc on prend 150x150 pixels
width=224
channel = 3
seed=1337
batch_size = 64
num_classes = 4

# Training generator
train_datagen = ImageDataGenerator(
        rescale=1./255, # Multiply the data by the rescale (after applying all other transformations)
        rotation_range=40, # Degree range for random rotations
        width_shift_range=0.2, # Fraction of total width
        height_shift_range=0.2, # Fraction of total height
        shear_range=0.2, # Shear Intensity
        zoom_range=0.2, # Range for random zoom
        horizontal_flip=True, # Randomly flip inputs horizontally
        fill_mode='nearest', # Points outside the boundaries of the input are filled according to 'aaaaaaaa|abcd|dddddddd'
        validation_split=0.2) # test

# train_datagen = ImageDataGenerator(preprocessing_function=keras.applications.VGG16.preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(height,width), # Size to resize images
                                                    batch_size=batch_size,
                                                    seed=seed,
                                                    shuffle=True,
                                                    class_mode='categorical', # class_mode='sparse' --> ordinal car tl le demande                             
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(train_dir, 
                                                         target_size=(height,width),
                                                         batch_size=batch_size,
                                                         seed=seed,
                                                         shuffle=True,
                                                         class_mode='categorical', # class_mode='categorical' --> one-hot  
                                                         subset='validation')

# Test generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                  target_size=(height,width),                                                   
                                                  batch_size=batch_size,
                                                  seed=seed,
                                                  shuffle=False,
                                                  class_mode='categorical') # class_mode='categorical' --> one-hot  

train_num = train_generator.samples
validation_num = validation_generator.samples
test_num = test_generator.samples 

Found 2297 images belonging to 4 classes.
Found 573 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [8]:
model = keras.models.Sequential([
                     keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channel)),
                     keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
                     keras.layers.Conv2D(32, (3, 3), activation='relu'),
                     keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
                     keras.layers.Conv2D(64, (3, 3), activation='relu'), # add
                     keras.layers.MaxPooling2D(pool_size=(2, 2)), # add
    
                     keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'), # 64
                     keras.layers.Conv2D(128, (3, 3), activation='relu'), # 64
                     keras.layers.MaxPooling2D(pool_size=(2, 2)),
                     keras.layers.Dropout(0.25),
                         
                     keras.layers.Flatten(),                     
                     keras.layers.Dense(256, activation='relu'), # 512
                     keras.layers.Dropout(0.5),
                     keras.layers.Dense(num_classes, activation='softmax')    
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc',
                                              patience=10, 
                                              restore_best_weights=True)

history = model.fit_generator(train_generator,
                              steps_per_epoch= train_num // batch_size,
                              epochs=200,
                              validation_data=validation_generator,
                              validation_steps= validation_num // batch_size,
                              callbacks=[early_stopping])#0.8232 --> 0.8625

Epoch 1/200
35/35 [==============================] - 35s 1s/step - loss: 1.3141 - acc: 0.3587 - val_loss: 1.5161 - val_acc: 0.3145
Epoch 2/200
35/35 [==============================] - 35s 1s/step - loss: 1.2275 - acc: 0.4411 - val_loss: 1.2438 - val_acc: 0.4538
Epoch 3/200
35/35 [==============================] - 34s 981ms/step - loss: 1.0857 - acc: 0.5207 - val_loss: 1.2185 - val_acc: 0.4637
Epoch 4/200
35/35 [==============================] - 34s 977ms/step - loss: 1.0227 - acc: 0.5598 - val_loss: 1.3512 - val_acc: 0.4401
Epoch 5/200
35/35 [==============================] - 34s 985ms/step - loss: 0.9703 - acc: 0.5880 - val_loss: 1.1322 - val_acc: 0.4971
Epoch 6/200
35/35 [==============================] - 34s 959ms/step - loss: 0.9508 - acc: 0.6049 - val_loss: 1.1010 - val_acc: 0.5226
Epoch 7/200
35/35 [==============================] - 34s 957ms/step - loss: 0.8871 - acc: 0.6244 - val_loss: 1.1999 - val_acc: 0.5422
Epoch 8/200
35/35 [==============================] - 33s 950ms/step 

In [9]:
model.evaluate_generator(test_generator, test_generator.samples) # [8.29, 0.5693] --> [6.72, 0.5719]

[6.723012447357178, 0.5718727707862854]

Transfert Learning VGG16

In [8]:
# Lien Kaggle : https://www.kaggle.com/madz2000/flowers-classification-using-vgg19-87-accuracy

pre_trained_model = keras.applications.VGG19(input_shape=(224,224,3),
                                             include_top=False,
                                             weights="imagenet")

#pre_trained_model.trainable = False

for layer in pre_trained_model.layers[:19]: #There are 19 layers in VGG19 model. So I freezed the weights of the first 18 layers except for the last layer so that they don't get updated during backpropagation.
    layer.trainable = False

model = keras.models.Sequential([pre_trained_model,
                                 keras.layers.MaxPool2D((2,2) , strides = 2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(num_classes , activation='softmax')])

model.compile(optimizer = "adam" , loss = 'categorical_crossentropy', metrics = ['acc'])

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience = 2, verbose=1, factor=0.1, min_lr=0.000001)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc',
                                              patience=4, 
                                              restore_best_weights=True)

history = model.fit_generator(train_generator,
                              steps_per_epoch= train_num // batch_size,
                              epochs=20,
                              validation_data=validation_generator,
                              validation_steps= validation_num // batch_size,
                              callbacks=[learning_rate_reduction,early_stopping])#0.8232 --> 0.8625 --> 0.6483 --> 0.8114

Epoch 1/20
35/35 [==============================] - 3027s 86s/step - loss: 1.4699 - acc: 0.3336 - val_loss: 1.2734 - val_acc: 0.3906
Epoch 2/20
35/35 [==============================] - 3811s 109s/step - loss: 1.0894 - acc: 0.4957 - val_loss: 1.1094 - val_acc: 0.4322
Epoch 3/20
35/35 [==============================] - 3966s 113s/step - loss: 0.9069 - acc: 0.6117 - val_loss: 0.9317 - val_acc: 0.5324
Epoch 4/20
35/35 [==============================] - 3346s 96s/step - loss: 0.7819 - acc: 0.6785 - val_loss: 0.8439 - val_acc: 0.6169
Epoch 5/20
35/35 [==============================] - 1487s 42s/step - loss: 0.7166 - acc: 0.7170 - val_loss: 0.9929 - val_acc: 0.6365
Epoch 6/20
35/35 [==============================] - 1626s 46s/step - loss: 0.5780 - acc: 0.7801 - val_loss: 0.6093 - val_acc: 0.7367
Epoch 7/20
35/35 [==============================] - 1533s 44s/step - loss: 0.5200 - acc: 0.8052 - val_loss: 0.7611 - val_acc: 0.7171
Epoch 8/20
35/35 [==============================] - 1417s 40s/step 

In [9]:
model.evaluate_generator(test_generator, test_generator.samples) # [1.6427531242370605, 0.6788932681083679]

[1.9205785989761353, 0.6482966542243958]